# OFAC ONLY

In [2]:
import pandas as pd

df = pd.read_csv("sdn_ofac.csv", names = ['ent_num', 'sdn_name', 'sdn_type', 'program', 'title', 'call_sign', 'vess_type', 'tonnage', 'grt', 'vess_flag', 'vess_owner', 'remarks'], header=None)

In [5]:
df.sample(5)

,ent_num,sdn_name,sdn_type,program,title,call_sign,vess_type,tonnage,grt,vess_flag,vess_owner,remarks
8334,29798,"SADIQABADI, Alireza",individual,SDGT] [IFSR,-0-,-0-,-0-,-0-,-0-,-0-,-0-,"DOB 1982; POB Tehran, Iran; nationality Iran; ..."
8576,30418,KAVEH SHARGH STEEL CO.,-0-,IRAN-EO13876,-0-,-0-,-0-,-0-,-0-,-0-,-0-,Website http://kes.co.ir; Additional Sanctions...
6565,25063,AL HARAM FOREIGN EXCHANGE CO. LTD,-0-,SDGT,-0-,-0-,-0-,-0-,-0-,-0-,-0-,Website www.arabisc-haram.com; Linked To: ISLA...
3169,15096,INFORMATION SYSTEMS IRAN,-0-,NPWMD] [IFSR,-0-,-0-,-0-,-0-,-0-,-0-,-0-,Additional Sanctions Information - Subject to ...
2169,11532,NAMCHONGANG TRADING CORPORATION,-0-,NPWMD,-0-,-0-,-0-,-0-,-0-,-0-,-0-,Secondary sanctions risk: North Korea Sanction...


In [52]:
individuals_df = df.loc[df['sdn_type']=='individual']#[['sdn_name', 'sdn_type', 'title', 'remarks']]

In [108]:
def process_individuals(indiv_df):
    def preprocess_name(name):
        name = name.lower()
        name = name.replace(",", '').replace(".",'')
        return name
    def split_remarks(line):
        splitted = line.split("; ")
        out=[]
        for x in splitted:
            out.append(x.lower())
        return out
    def get_dob(attr_list):
        out = ''
        for x in attr_list:
            if 'dob' in x:
                splitted = x.split(' ')
                for i in range(len(splitted)):
                    if i == 1:
                        out += splitted[i]
                    elif i>1:
                        out += ' ' + splitted[i]
        if out == '':
            return None
        return out
    def get_pob(attr_list):
        out = ''
        for x in attr_list:
            if 'pob' in x:
                splitted = x.split(' ')
                for i in range(len(splitted)):
                    if i == 1:
                        out += splitted[i]
                    elif i>1:
                        out += ' ' + splitted[i]
        if out == '':
            return None
        return out
    def get_nationality(attr_list):
        out = ''
        for x in attr_list:
            if 'nationality' in x:
                splitted = x.split(' ')
                for i in range(len(splitted)):
                    if i == 1:
                        out += splitted[i]
                    elif i>1:
                        out += ' ' + splitted[i]
        if out == '':
            return None
        return out
    def get_gender(attr_list):
        out = ''
        for x in attr_list:
            if 'gender' in x:
                splitted = x.split(' ')
                for i in range(len(splitted)):
                    if i == 1:
                        out += splitted[i]
                    elif i>1:
                        out += ' ' + splitted[i]
        if out == '':
            return None
        return out
    def get_alias(attr_list):
        out = []
        for x in attr_list:
            if 'a.k.a.' in x:
                splitted = x.split(' ')
                for i in range(len(splitted)):
                    if i >= 1:
                        out.append(splitted[i].replace("'", '').replace('.', ''))
        if len(out)==0:
            return None
        return out
    def get_citizenship(attr_list):
        out = ''
        for x in attr_list:
            if 'citizen' in x:
                splitted = x.split(' ')
                for i in range(len(splitted)):
                    if i == 1:
                        out += splitted[i]
                    elif i>1:
                        out += ' ' + splitted[i]
        if out == '':
            return None
        return out
    df_dict = indiv_df.to_dict('records')
    output = []
    for record in df_dict:
        name = preprocess_name(record['sdn_name'])
        remarks = split_remarks(record['remarks'])
        dob = get_dob(remarks)
        pob = get_pob(remarks)
        nationality = get_nationality(remarks)
        gender = get_gender(remarks)
        alias_list = get_alias(remarks)
        citizenship = get_citizenship(remarks)
        title = record['title']
        current_dict = {
            'name' : name,
            'title' : title if title != '-0- ' else None, 
            'gender' : gender,
            'dob': dob,
            'pob': pob,
            'nationality' : nationality,
            'citizenship' : citizenship,
            'aliases' : alias_list
        }
        output.append(current_dict)
    return output

In [111]:
process_indiv_df = process_individuals(individuals_df)
process_indiv_df = pd.DataFrame(process_indiv_df)

In [112]:
process_indiv_df.to_csv("ofac_individuals.csv")

In [14]:
# companies_df = df.loc[df['sdn_type']!='individual']

In [120]:
# companies_df.iloc[601]

ent_num                                                   10249
sdn_name                      COMERCIALIZADORA MOR GAVIRIA S.A.
sdn_type                                                   -0- 
program                                                    SDNT
title                                                      -0- 
call_sign                                                  -0- 
vess_type                                                  -0- 
tonnage                                                    -0- 
grt                                                        -0- 
vess_flag                                                  -0- 
vess_owner                                                 -0- 
remarks       RUC # 1791813359001 (Ecuador); a.k.a. 'DURATEX...
Name: 1576, dtype: object

# Cleaning up Consolidated Individuals Sanction List

In [4]:

import pandas as pd
df = pd.read_csv("consolidated_sanction_list.csv")[['name', 'title', 'dob','pob','nationality','citizenship','aliases']]

In [9]:
len(df)

15738

In [77]:
df = df[df['name'].notna()]
len(df)

12095

In [92]:
from deep_translator import GoogleTranslator

def process_names(df):
    def clean_name(name):
        name = name.lower()
        name = name.replace(",", '').replace(".",'').replace('"','').replace("'",'').replace("`",'')
        return name
    
    def translate_name(name):
        translated = GoogleTranslator(source='auto', target='en').translate(name)
        return translated

    def isEnglish(s):
            try:
                s.encode(encoding='utf-8').decode('ascii')
            except UnicodeDecodeError:
                return False
            else:
                return True 
    df_dict = df.to_dict('records')
    output = []
    counter=0
    for record in df_dict:
        counter+=1
        if counter == 5000:
            counter=0
            print("5000")
        name = record['name']
        name = clean_name(name)
        flag = isEnglish(name)
        if flag is False:
            name = translate_name(name).lower()
        record['name'] = name
        output.append(record)
    return output

In [93]:
df = process_names(df)

5000
5000


In [94]:
df = pd.DataFrame(df)

In [95]:
df.to_csv("cleaned_indiv_sanction_list.csv")

In [96]:
df

,name,title,dob,pob,nationality,citizenship,aliases
0,noriega manuel antonio,NaN,NaN,NaN,NaN,NaN,NaN
1,sieiro de noriega felicidad,NaN,NaN,NaN,NaN,NaN,NaN
2,abbas abu,Director of PALESTINE LIBERATION FRONT - ABU A...,10-Dec-48,NaN,NaN,NaN,NaN
3,al rahman shaykh umar abd,Chief Ideological Figure of ISLAMIC GAMA'AT,3-May-38,egypt,NaN,NaN,NaN
4,al zawahiri dr ayman,Operational and Military Leader of JIHAD GROUP,19-Jun-51,"giza, egypt",NaN,NaN,NaN
...,...,...,...,...,...,...,...
12090,ana julia guido ochoa,Attorney General,NaN,,NaN,NaN,NaN
12091,fidel de jesús domínguez alvarez,chief of the police in Leon since 23 August 2018,NaN,,NaN,NaN,NaN
12092,alba luz ramos vanegas,President of the Supreme Court of Justice of N...,NaN,,NaN,NaN,NaN
12093,juan carlos ortega murillo,Director at Canal 8 and Difuso Comunicaciones....,NaN,,NaN,NaN,NaN
